In [59]:
import numpy as np

In [60]:
def append_composite_dictionary(composite_dict, models):
    for model in models:
        composite_dict.update(model.param_dict)

In [61]:
def update_component_dictionaries(composite_dict, models):
    for model in models:
        for key in model.param_dict.keys():
            model.param_dict[key] = composite_dict[key]

In [62]:
class Intrinsicmodel(object):
    def __init__(self, sample_name, **kwargs):
        self.param_dict = {}
        self.sample_name = sample_name
        self.modelname = 'intrinsic_Allignement_model_'+self.sample_name
        self.param_dict['A1_'+self.sample_name] = 0.5
        self.param_dict['A2_'+self.sample_name] = 2.
        
    def intrinsic_bias_strength(self):
        return self.param_dict['A1'+self.sample_name]+self.param_dict['A2'+self.sample_name]

In [63]:
class Multiplicative_bias_model(object):
    def __init__(self, sample_name, **kwargs):
        self.param_dict = {}
        self.sample_name = sample_name
        self.modelname = 'Multiplicative_bias_model_'+self.sample_name
        self.param_dict['multiplicative_bias_'+self.sample_name] = 1.5
        
    def multiplicative_bias_strength(self):
        return self.param_dict['multiplicative_bias_'+self.sample_name]

In [64]:
class source_galaxy(object):
    def __init__(self, sample_name, **kwargs):
        self.sample_name = sample_name
        self.param_dict = {}
        #self.param_dict['intrinsic_model_'+self.sample_name] = IA_model
        #self.param_dict['A1_'+self.sample_name,'A2_'+self.sample_name] = A1, A2
        #self.param_dict['multiplicative_bias_model_'+self.sample_name] = multiplicative_bias_model
        #self.param_dict['multiplicative_bias_'+self.sample_name] = multiplicative_bias
        self.param_dict['n_of_z_'+self.sample_name] = kwargs['n_of_z']
        
        try:
            self.models = kwargs['models']
        except KeyError:
            self.models = []
            
        for model in self.models:
            setattr(self, model.modelname, model)
        append_composite_dictionary(self.param_dict, self.models)
        
    def intrinsic_alignments(self, *args):
        for model in self.models:
            for key in model.param_dict.keys():
                model.param_dict[key] = self.param_dict[key]
                
        return self.param_dict['n_of_z_'+self.sample_name]*(self.param_dict['A1_'+self.sample_name]-self.param_dict['A2_'+self.sample_name])
        
    def multiplicative_bias(self, *args):
        return self.intrinsic_alignments * self.param_dict['multiplicative_bias_'+self.sample_name]

In [106]:
class ShearShearDataVector(object):
    def __init__(self, source1, source2, **kwargs):
        #additive_bias = kwargs['additive_bias']
        #atmospheric_power_model = kwargs['atmospheric_power_model']
        # not defined here probably: intrinsic_alignment = kwargs['intrinsic alignment']
        
        #self?
        try:
            self.models = kwargs['models']
        except KeyError:
            self.models = []

        self.param_dict = {}
        append_composite_dictionary(self.param_dict, self.models)
        append_composite_dictionary(self.param_dict, [source1, source2])
        self.source1 = source1
        self.source2 = source2
        self.source1.name = source1.sample_name
        self.source2.name = source2.sample_name
        
        
        update_component_dictionaries(self.param_dict, [self.source1,self.source2])
    def C_ell_GG(self):
        ## where is cosmology specified??
        n_of_z_1 = self.source1.param_dict['n_of_z_'+self.source1.name]
        n_of_z_2 = self.source2.param_dict['n_of_z_'+self.source2.name]
        return n_of_z_1 + n_of_z_2 #some CCL call
        
    def C_ell_GI(self):
        n_of_z_1 = self.source1.param_dict['n_of_z_'+self.source1.name]
        n_of_z_2 = self.source2.param_dict['n_of_z_'+self.source2.name]
        A_1_1 = self.source1.param_dict['A1_'+self.source1.name]
        A_1_2 = self.source2.param_dict['A1_'+self.source2.name]
        return A_1_1*n_of_z_1 + n_of_z_2 #some CCL call
    
    def C_ell_IG(self):
        n_of_z_1 = self.source1.param_dict['n_of_z_'+self.source1.name]
        n_of_z_2 = self.source2.param_dict['n_of_z_'+self.source2.name]
        A_1_1 = self.source1.param_dict['A1_'+self.source1.name]
        A_1_2 = self.source2.param_dict['A1_'+self.source2.name]
        return n_of_z_1 + A_1_2*n_of_z_2 #some CCL call
    
    def C_ell_II(self):
        n_of_z_1 = self.source1.param_dict['n_of_z_'+self.source1.name]
        n_of_z_2 = self.source2.param_dict['n_of_z_'+self.source2.name]
        A_1_1 = self.source1.param_dict['A1_'+self.source1.name]
        A_1_2 = self.source2.param_dict['A1_'+self.source2.name]
        return A_1_1*n_of_z_1 + A_1_2*n_of_z_2 #some CCL call

In [107]:
intrinsic1 = Intrinsicmodel('first')
multbias1 = Multiplicative_bias_model('first')
first_galaxy = source_galaxy('first', models=[intrinsic1, multbias1], n_of_z = 0.2)

intrinsic2 = Intrinsicmodel('second')
multbias2 = Multiplicative_bias_model('second')
second_galaxy = source_galaxy('second', models=[intrinsic2, multbias2], n_of_z = 0.3)

In [108]:
first_galaxy.param_dict

{'A1_first': 0.5,
 'A2_first': 2.0,
 'multiplicative_bias_first': 1.5,
 'n_of_z_first': 0.2}

In [109]:
first_galaxy.intrinsic_alignments()

-0.30000000000000004

In [110]:
second_galaxy.intrinsic_alignments()

-0.44999999999999996

In [111]:
ss = ShearShearDataVector(first_galaxy, second_galaxy)

In [112]:
ss.C_ell_GG()

0.5

In [113]:
ss.param_dict

{'A1_first': 0.5,
 'A1_second': 0.5,
 'A2_first': 2.0,
 'A2_second': 2.0,
 'multiplicative_bias_first': 1.5,
 'multiplicative_bias_second': 1.5,
 'n_of_z_first': 0.2,
 'n_of_z_second': 0.3}

In [115]:
ss.C_ell_II()

0.25

In [116]:
ss.C_ell_IG()

0.35

In [117]:
ss.C_ell_GI()

0.4